# 環境構築

In [1]:
!git clone https://github.com/marukun712/MRTalk-GPT-SoVITS.git
%cd MRTalk-GPT-SoVITS

Cloning into 'MRTalk-GPT-SoVITS'...
remote: Enumerating objects: 4688, done.
remote: Total 4688 (delta 0), reused 0 (delta 0), pack-reused 4688 (from 1)
Receiving objects: 100% (4688/4688), 13.16 MiB | 3.63 MiB/s, done.
Resolving deltas: 100% (2703/2703), done.
Updating files: 100% (300/300), done.
/content/MRTalk-GPT-SoVITS


In [2]:
!uv pip install -q -r requirements.txt
!uv pip install -q ffmpeg-python pyopenjtalk pyngrok

In [3]:
%%bash
cat <<EOF >  /content/MRTalk-GPT-SoVITS/GPT_SoVITS/configs/tts_infer.yaml
custom:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cuda
  is_half: true
  t2s_weights_path: GPT_SoVITS/pretrained_models/s1v3.ckpt
  version: v4
  vits_weights_path: GPT_SoVITS/pretrained_models/gsv-v4-pretrained/s2Gv4.pth
v1:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cpu
  is_half: false
  t2s_weights_path: GPT_SoVITS/pretrained_models/s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt
  version: v1
  vits_weights_path: GPT_SoVITS/pretrained_models/s2G488k.pth
v2:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cpu
  is_half: false
  t2s_weights_path: GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s1bert25hz-5kh-longer-epoch=12-step=369668.ckpt
  version: v2
  vits_weights_path: GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s2G2333k.pth
v3:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cpu
  is_half: false
  t2s_weights_path: GPT_SoVITS/pretrained_models/s1v3.ckpt
  version: v3
  vits_weights_path: GPT_SoVITS/pretrained_models/s2Gv3.pth
v4:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cpu
  is_half: false
  t2s_weights_path: GPT_SoVITS/pretrained_models/s1v3.ckpt
  version: v4
  vits_weights_path: GPT_SoVITS/pretrained_models/gsv-v4-pretrained/s2Gv4.pth
EOF

In [5]:
!rm ./GPT_SoVITS/pretrained_models/.gitignore
!git clone https://huggingface.co/lj1995/GPT-SoVITS ./GPT_SoVITS/pretrained_models/

Cloning into './GPT_SoVITS/pretrained_models'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 53 (delta 9), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (53/53), 111.70 KiB | 2.54 MiB/s, done.
Filtering content: 100% (13/13), 3.49 GiB | 29.96 MiB/s, done.


# Launch Server
Google Colabのシークレット環境変数(左側のタブにあります)にNGROK_TOKEN, NGROK_DOMAIN, DATABASE_URLを追加する必要があります。  
https://ngrok.com/

In [6]:
from google.colab import userdata
from pyngrok import ngrok

TOKEN = userdata.get("NGROK_TOKEN")
DOMAIN = userdata.get("NGROK_DOMAIN")
DATABASE_URL = userdata.get("DATABASE_URL")
%env DATABASE_URL=$DATABASE_URL

ngrok.set_auth_token(TOKEN)

ngrok_tunnel = ngrok.connect(9880, domain=DOMAIN)
print('PUBLIC_URL:',ngrok_tunnel.public_url)

PUBLIC_URL: https://eagle-cosmic-perfectly.ngrok-free.app


In [7]:
!nohup python api_v2.py -a 127.0.0.1 -p 9880 -c GPT_SoVITS/configs/tts_infer.yaml > server.log 2>&1 &

# Demo
Colab上から動かせるようにしました。  
text: 読み上げ文章  
ref_audio_path: 学習/参照する声のパス(currentpathはMRTalk-GPT-SoVITSです)  
prompt_text: ref_audioが読み上げている文章  


In [ ]:
# ref_audio.wavにサンプル音声を入れてください。
import requests

PATH = "http://127.0.0.1:9880"

def tts_get():
    # GETリクエストのパラメータを設定
    params = {
        'text': 'てきすと',
        'text_lang': 'ja',
        'ref_audio_path': 'ref_audio.wav',
        'prompt_text': '入力プロンプト',
        'prompt_lang': 'ja',
        'media_type': 'wav',
        'streaming_mode': 'false'
    }

    # GETリクエストを送信
    response = requests.get(PATH + '/tts', params=params)

    # レスポンスの処理
    if response.status_code == 200:
        # 音声データをファイルに保存
        with open('output_get.wav', 'wb') as f:
            f.write(response.content)
        print('音声ファイルをoutput_get.wavに保存しました。')
    else:
        print(f"エラー {response.status_code}: {response.text}")

tts_get()

音声ファイルをoutput_get.wavに保存しました。


In [ ]:
!cat server.log